In [1]:
import osmnx as ox
import pandas as pd
import geopandas as gpd

from rapidfuzz import process, fuzz

import positron as pt

pd.set_option('display.max_columns', None)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read all parts and concat
df = pd.read_csv('./data/raw/transactions.csv')

### 1. Removing Unneccessary Columns & Cleaning

In [20]:
df.drop(columns=[col for col in df.columns if col.endswith('_ar')], inplace=True)
df.rename(columns=lambda x: x[:-3] if x.endswith('_en') else x, inplace=True)

df.drop(columns=['project_number', 'project_name', 'master_project', 'rent_value', 'meter_rent_price', 'no_of_parties_role_1', 'no_of_parties_role_2', 'no_of_parties_role_3'], inplace=True)
df.rename(columns={'procedure_area':'procedure_area_sqm'}, inplace=True)

In [21]:
df['instance_date'] = pd.to_datetime(df['instance_date'], dayfirst=True, errors='coerce')

In [7]:
df['trans_group'] = df['trans_group'].str.lower().str.strip()
df['procedure_name'] = df['procedure_name'].str.lower().str.strip()
df['property_type'] = df['property_type'].str.lower().str.strip()
df['property_sub_type'] = df['property_sub_type'].str.lower().str.strip()
df['property_usage'] = df['property_usage'].str.lower().str.strip()
df['reg_type'] = df['reg_type'].str.lower().str.strip()
df['area_name'] = df['area_name'].str.lower().str.strip()
df['building_name'] = df['building_name'].str.lower().str.strip()
df['nearest_landmark'] = df['nearest_landmark'].str.lower().str.strip()
df['nearest_metro'] = df['nearest_metro'].str.lower().str.strip()
df['nearest_mall'] = df['nearest_mall'].str.lower().str.strip()

In [8]:
# Currency conversion (exchange rate as of 02.12.2025)
df['actual_worth_usd'] = df['actual_worth'] * 0.27
df['meter_sale_price_usd'] = df['meter_sale_price'] * 0.27

df.rename(columns={'actual_worth': 'actual_worth_aed', 'meter_sale_price':'meter_sale_price_aed'}, inplace=True)

In [9]:
# Calculate price-to-size ratio
df.loc[:, 'price_to_size_ratio'] = (df['meter_sale_price_usd'] / df['procedure_area_sqm']).round(2)

In [10]:
df

,transaction_id,procedure_id,trans_group_id,trans_group,procedure_name,instance_date,property_type_id,property_type,property_sub_type_id,property_sub_type,property_usage,reg_type_id,reg_type,area_id,area_name,building_name,nearest_landmark,nearest_metro,nearest_mall,rooms,has_parking,procedure_area_sqm,actual_worth_aed,meter_sale_price_aed,actual_worth_usd,meter_sale_price_usd,price_to_size_ratio
0,1-11-2024-10138,11,1,sales,sell,2024-03-19,4,villa,NaN,NaN,residential,1,existing properties,278,mankhool,NaN,burj khalifa,adcb metro station,dubai mall,NaN,0,1305.29,5769000.0,4419.71,1557630.00,1193.3217,0.91
1,3-9-2002-39,9,3,gifts,grant,2002-03-25,1,land,NaN,NaN,residential,1,existing properties,365,al hudaiba,NaN,burj khalifa,adcb metro station,dubai mall,NaN,0,1466.94,1105300.0,753.47,298431.00,203.4369,0.14
2,1-11-2016-12930,11,1,sales,sell,2016-11-02,4,villa,NaN,NaN,residential,1,existing properties,276,al bada,NaN,burj khalifa,emirates towers metro station,dubai mall,NaN,0,390.00,2089900.0,5358.72,564273.00,1446.8544,3.71
3,1-11-2005-300028,11,1,sales,sell,2005-02-28,4,villa,NaN,NaN,residential,1,existing properties,276,al bada,NaN,burj khalifa,trade centre metro station,dubai mall,NaN,0,396.09,511612.0,1291.66,138135.24,348.7482,0.88
4,1-11-2013-7839,11,1,sales,sell,2013-03-12,4,villa,NaN,NaN,residential,1,existing properties,276,al bada,NaN,burj khalifa,emirates towers metro station,dubai mall,NaN,0,362.32,1500000.0,4139.99,405000.00,1117.7973,3.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397737,1-102-2025-9135,102,1,sales,sell - pre registration,2025-02-06,3,unit,60.0,flat,residential,0,off-plan properties,330,marsa dubai,bayview tower 2,NaN,NaN,NaN,1 B/R,1,70.72,3443888.0,48697.51,929849.76,13148.3277,185.92
1397738,1-102-2023-45846,102,1,sales,sell - pre registration,2023-09-11,3,unit,60.0,flat,residential,0,off-plan properties,330,marsa dubai,bayview tower 1,NaN,NaN,NaN,1 B/R,1,69.85,3664888.0,52467.97,989519.76,14166.3519,202.81
1397739,1-102-2024-879,102,1,sales,sell - pre registration,2024-01-08,3,unit,60.0,flat,residential,0,off-plan properties,330,marsa dubai,bayview tower 2,NaN,NaN,NaN,1 B/R,1,70.72,3705888.0,52402.26,1000589.76,14148.6102,200.07
1397740,1-102-2023-46068,102,1,sales,sell - pre registration,2023-09-11,3,unit,60.0,flat,residential,0,off-plan properties,330,marsa dubai,bayview tower 2,NaN,NaN,NaN,1 B/R,1,53.16,2430888.0,45727.77,656339.76,12346.4979,232.25


### 2. Dealing with Outliers

In [11]:
df[df['property_type'] == 'land']['actual_worth_usd'].describe([0.25, 0.5, 0.75, 0.9, 0.95, 0.98, 0.99, 0.999])

count    1.200590e+05
mean     4.221879e+06
std      5.613071e+07
min      2.700000e-01
25%      3.837274e+05
50%      7.025400e+05
75%      1.878090e+06
90%      5.156676e+06
95%      9.180000e+06
98%      2.025000e+07
99%      3.780000e+07
99.9%    3.833212e+08
max      3.722473e+09
Name: actual_worth_usd, dtype: float64

In [12]:
# CHOPPING THE TOP 1 PERCENT

unit_outliers = df[(df['property_type'] == 'unit') & (df['actual_worth_usd'] > 2000000)]
villa_outliers = df[(df['property_type'] == 'villa') & (df['actual_worth_usd'] > 2800000)]
building_outliers = df[(df['property_type'] == 'building') & (df['actual_worth_usd'] > 30000000)]
land_outliers = df[(df['property_type'] == 'land') & (df['actual_worth_usd'] > 20000000)]

outliers = pd.concat([unit_outliers, villa_outliers, building_outliers, land_outliers], ignore_index=True)

In [13]:
outliers

,transaction_id,procedure_id,trans_group_id,trans_group,procedure_name,instance_date,property_type_id,property_type,property_sub_type_id,property_sub_type,property_usage,reg_type_id,reg_type,area_id,area_name,building_name,nearest_landmark,nearest_metro,nearest_mall,rooms,has_parking,procedure_area_sqm,actual_worth_aed,meter_sale_price_aed,actual_worth_usd,meter_sale_price_usd,price_to_size_ratio
0,1-11-2012-7375,11,1,sales,sell,2012-04-24,3,unit,23.0,shop,commercial,1,existing properties,341,trade center first,latifa tower,burj khalifa,emirates towers metro station,dubai mall,Shop,1,573.48,27778005.0,48437.62,7500061.35,13078.1574,22.80
1,1-102-2023-6642,102,1,sales,sell - pre registration,2023-02-10,3,unit,60.0,flat,residential,0,off-plan properties,371,al safouh second,the s tower,burj al arab,dubai internet city,marina mall,4 B/R,1,447.24,15036984.0,33621.73,4059985.68,9077.8671,20.30
2,1-11-2015-13569,11,1,sales,sell,2015-09-02,3,unit,23.0,shop,commercial,1,existing properties,330,marsa dubai,azure,burj al arab,jumeirah lakes towers,marina mall,NaN,1,353.40,7750000.0,21929.82,2092500.00,5921.0514,16.75
3,1-11-2011-3369,11,1,sales,sell,2011-02-24,3,unit,42.0,office,commercial,1,existing properties,351,al thanyah third,emmar business park bldg-4,burj al arab,dubai internet city,marina mall,Office,1,2393.18,11428571.0,4775.48,3085714.17,1289.3796,0.54
4,1-11-2024-41555,11,1,sales,sell,2024-10-31,3,unit,60.0,flat,residential,1,existing properties,410,palm jumeirah,the residences north,burj al arab,palm jumeirah,marina mall,PENTHOUSE,1,429.71,15000000.0,34907.26,4050000.00,9424.9602,21.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26945,2-14-2024-1659,14,2,mortgages,modify mortgage,2024-12-24,1,land,NaN,NaN,commercial,1,existing properties,445,jabal ali first,NaN,expo 2020 site,energy metro station,ibn-e-battuta mall,NaN,0,34421.09,182250000.0,5294.72,49207500.00,1429.5744,0.04
26946,1-11-2024-26991,11,1,sales,sell,2024-07-22,1,land,NaN,NaN,commercial,1,existing properties,444,al hebiah first,NaN,motor city,sharaf dg metro station,mall of the emirates,NaN,0,15877.89,93921450.0,5915.23,25358791.50,1597.1121,0.10
26947,1-41-2015-7987,41,1,sales,delayed sell,2016-02-08,1,land,NaN,NaN,residential,1,existing properties,412,al merkadh,NaN,downtown dubai,business bay metro station,dubai mall,NaN,0,3370.46,91952320.0,27281.83,24827126.40,7366.0941,2.19
26948,1-41-2022-2974,41,1,sales,delayed sell,2022-02-22,1,land,NaN,NaN,multi-use,1,existing properties,410,palm jumeirah,NaN,NaN,NaN,NaN,NaN,0,20530.42,240000000.0,11689.97,64800000.00,3156.2919,0.15


In [14]:
df = df[~df['transaction_id'].isin(outliers['transaction_id'])]

In [15]:
df.to_csv('./data/processed/transactions_clean.csv', index=0)

### 3. Finding the Area Plygons

In [128]:
df = pd.read_parquet('./data/processed/transactions_clean.parquet')
areas = gpd.read_file('./data/raw/dubai_land_areas.geojson')

In [129]:
df['area_name'] = df['area_name'].str.lower().str.strip()
areas['CNAME_E'] = areas['CNAME_E'].str.lower().str.strip()

In [130]:
df['actual_worth_aed'] = df['actual_worth_aed'] / 1000000
df['actual_worth_usd'] = df['actual_worth_usd'] / 1000000

#### Approach 1 of merging (names)

In [ ]:
areas_names = areas['CNAME_E'].unique()

# Function to get the best match for each area_name
def match_area_name(area_name, choices, threshold=95):
    match, score, _ = process.extractOne(area_name, choices, scorer=fuzz.WRatio)
    if score >= threshold:
        return match
    else:
        return None

df['matched_area'] = df['area_name'].apply(lambda x: match_area_name(x, areas_names))

In [6]:
areas = areas[areas['CNAME_E'].isin(df['matched_area'])]

In [146]:
areas = areas[['COMM_NUM', 'Name', 'SHAPE.AREA', 'geometry']]

areas['Name'] = areas['Name'].str.lower().str.strip()
areas.rename(columns={'Name':'name', 'COMM_NUM':'comm_number', 'SHAPE.AREA':'area_sqm'}, inplace=True)

In [ ]:
areas = areas.merge(df[['matched_area', 'area_id']], left_on='name', right_on='matched_area')

areas.drop_duplicates(inplace=True)
areas.reset_index(inplace=True)

areas = areas[['area_id', 'comm_number', 'name', 'area_sqm', 'geometry']]

#### Approach 2 of merging (manually created area_id)

In [131]:
areas = areas[areas['area_id'].notna()]

areas = areas[['area_id', 'COMM_NUM', 'Name', 'SHAPE.AREA', 'geometry']]

areas['Name'] = areas['Name'].str.lower().str.strip()
areas.rename(columns={'Name':'name', 'COMM_NUM':'comm_number', 'SHAPE.AREA':'area_sqm'}, inplace=True)

In [132]:
areas

,area_id,comm_number,name,area_sqm,geometry
0,453.0,531.0,saih shuaib 2,2.610625e+07,"POLYGON ((55.00949 24.85906, 55.04427 24.83957..."
1,410.0,381.0,nakhlat jumeira,2.674180e+07,"POLYGON ((55.14542 25.09321, 55.14590 25.09281..."
2,368.0,373.0,al barsha first,3.954266e+06,"POLYGON ((55.20902 25.11838, 55.20817 25.11895..."
3,435.0,682.0,al hebiah fourth,9.145534e+06,"POLYGON ((55.22886 25.03974, 55.22665 25.04414..."
5,451.0,683.0,al hebiah fifth,8.093324e+06,"POLYGON ((55.25201 25.01084, 55.24750 25.01573..."
...,...,...,...,...,...
218,398.0,711.0,al awir first,3.971613e+07,"POLYGON ((55.63303 25.19255, 55.62934 25.21013..."
219,323.0,248.0,al qusais ind. fifth,2.395555e+06,"POLYGON ((55.42775 25.28670, 55.42681 25.28704..."
220,402.0,265.0,oud al muteena first,3.628021e+06,"POLYGON ((55.46027 25.27242, 55.45839 25.27288..."
221,524.0,857.0,margab,3.463383e+07,"POLYGON ((55.64260 24.78481, 55.61671 24.85200..."


#### Adding the total worth of the area for each property type (i.e. unit, villa, land, etc.)

In [ ]:
def add_property_type_worth(df, areas):
    worth_by_area_type = df.groupby(['area_id', 'property_type'])['actual_worth_usd'].sum().reset_index()

    worth_pivot = worth_by_area_type.pivot(index='area_id', columns='property_type', values='actual_worth_usd').fillna(0)
    worth_pivot = worth_pivot.reset_index()

    worth_pivot = worth_pivot.rename(columns=lambda x: f'{x.lower()}_total_worth_usd_millions' if x != 'area_id' else x)

    areas_merged = areas.merge(worth_pivot, how='left', on='area_id')

    for col in worth_pivot.columns:
        if col != 'area_id':
            areas_merged[col] = areas_merged[col].fillna(0)
    
    total_worth = (
        df.groupby('area_id')['actual_worth_usd']
        .sum()
        .reset_index()
        .rename(columns={'actual_worth_usd': 'total_worth_usd_millions'})
    )

    areas_merged = areas_merged.merge(total_worth, how='left', on='area_id')
    areas_merged['total_worth_usd_millions'] = areas_merged['total_worth_usd_millions'].fillna(0)

    return areas_merged

def add_property_type_counts(df, areas):
    counts = (
        df.groupby(['area_id', 'property_type'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

    # Merge with areas
    areas_merged = areas.merge(counts, how='left', on='area_id')

    # Fill NaNs for counts
    for col in ['number_of_lands', 'number_of_units', 'number_of_villas', 'number_of_buildings']:
        if col in areas_merged.columns:
            areas_merged[col] = areas_merged[col].fillna(0).astype(int)

    return areas_merged

In [134]:
areas = add_property_type_worth(df, areas)
areas = add_property_type_counts(df, areas)

In [135]:
areas

,area_id,comm_number,name,area_sqm,geometry,building_total_worth_usd_millions,land_total_worth_usd_millions,unit_total_worth_usd_millions,villa_total_worth_usd_millions,total_worth_usd_millions,building,land,unit,villa
0,453.0,531.0,saih shuaib 2,2.610625e+07,"POLYGON ((55.00949 24.85906, 55.04427 24.83957...",718.498395,2061.573608,80.733924,131.964364,2992.770291,108.0,601.0,483.0,438.0
1,410.0,381.0,nakhlat jumeira,2.674180e+07,"POLYGON ((55.14542 25.09321, 55.14590 25.09281...",781.595611,4552.555155,20711.272568,4938.248851,30983.672184,49.0,1536.0,30622.0,3370.0
2,368.0,373.0,al barsha first,3.954266e+06,"POLYGON ((55.20902 25.11838, 55.20817 25.11895...",8504.423309,2103.456455,100.629835,112.151759,10820.661357,947.0,378.0,367.0,83.0
3,435.0,682.0,al hebiah fourth,9.145534e+06,"POLYGON ((55.22886 25.03974, 55.22665 25.04414...",1024.202125,6341.834272,5606.325804,6721.344959,19693.707159,147.0,2540.0,29385.0,8140.0
4,451.0,683.0,al hebiah fifth,8.093324e+06,"POLYGON ((55.25201 25.01084, 55.24750 25.01573...",45.335454,8448.710631,3100.093374,256.921808,11851.061267,5.0,11055.0,13123.0,268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,398.0,711.0,al awir first,3.971613e+07,"POLYGON ((55.63303 25.19255, 55.62934 25.21013...",155.876962,1178.130321,0.000000,6.581451,1340.588734,70.0,3910.0,0.0,21.0
186,323.0,248.0,al qusais ind. fifth,2.395555e+06,"POLYGON ((55.42775 25.28670, 55.42681 25.28704...",253.350759,607.700590,28.725144,0.000000,889.776493,44.0,98.0,158.0,0.0
187,402.0,265.0,oud al muteena first,3.628021e+06,"POLYGON ((55.46027 25.27242, 55.45839 25.27288...",24.651000,143.622352,0.000000,175.152555,343.425907,5.0,415.0,0.0,847.0
188,524.0,857.0,margab,3.463383e+07,"POLYGON ((55.64260 24.78481, 55.61671 24.85200...",15.955897,7.846120,0.000000,0.000000,23.802017,12.0,4.0,0.0,0.0


In [152]:
areas['worth_per_sqm'] = (areas['total_worth_usd_millions'] * 1000000) / (areas['area_sqm'])

areas['unit_worth_per_sqm'] = (areas['unit_total_worth_usd_millions'] * 1000000) / (areas['area_sqm'])
areas['villa_worth_per_sqm'] = (areas['villa_total_worth_usd_millions'] * 1000000) / (areas['area_sqm'])
areas['building_worth_per_sqm'] = (areas['building_total_worth_usd_millions'] * 1000000) / (areas['area_sqm'])
areas['land_worth_per_sqm'] = (areas['land_total_worth_usd_millions'] * 1000000) / (areas['area_sqm'])

In [154]:
areas['properties_count'] = areas['unit'] + areas['villa'] + areas['building'] + areas['land']

In [156]:
areas.to_file('./data/processed/transaction_land_areas.geojson', driver='GeoJSON')

### 4. Metro Stations Cleaning

In [472]:
stations = gpd.read_file('./data/raw/dubai_metro_stations.geojson')

In [474]:
stations.drop(columns=['LAT_STATION', 'LON_STATION', 'ROUTE'], inplace=True)

In [477]:
stations.columns = [col.lower() for col in stations.columns]

for col in stations.columns:
    if  isinstance(stations[col][0], str):
        stations[col] = stations[col].str.strip().str.lower()

In [479]:
stations.to_file('./data/processed/dubai_metro_stations.geojson', driver='GeoJSON')

In [506]:
venues

,comm_num,venue_id,venue,rating,venue_type,geometry
0,111,4ccafdcaaa25a35d0e100b0f,Miyako 京,8.7,restaurant/cafe/bar,POINT (55.30434 25.27892)
1,111,4d76451348b7f04dabadf2f5,Al Dawaar,8.5,restaurant/cafe/bar,POINT (55.30441 25.27911)
2,111,4c0190f4b58376b05062443c,The Kitchen,8.3,restaurant/cafe/bar,POINT (55.30427 25.27905)
3,111,4df4e8d9aeb7170aa2f5e7c5,Club Olympus,8.2,fitness center,POINT (55.30444 25.27836)
5,111,508ebc87e4b0d79b8faf6a9c,Sharzad,7.5,restaurant/cafe/bar,POINT (55.30425 25.27902)
...,...,...,...,...,...,...
2919,931,56ac945b498e8dd1ca5f0c2c,Hum Yum هم يم,NaN,restaurant/cafe/bar,POINT (55.49416 24.94974)
2922,941,4f92a4abe4b04c3ff9332887,مطعم الرمان,7.0,restaurant/cafe/bar,POINT (55.55677 24.86491)
2928,961,59a2ece8535d6f7229d3bdae,Last Exit D63,7.1,fast food,POINT (55.34344 24.85405)
2939,971,5d9797dd620790000841232b,Bamboo Meadow,NaN,shop/market,POINT (55.47486 24.71886)
